In [6]:
# Imports and configuration
%load_ext autoreload
%autoreload 2

import src.utils.file_utils as fu
import src.helpers.feature_engineering_helper as feh

cfg_schema = fu.load_config("schema")
cfg_feature_engineering = fu.load_config("feature_engineering")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# Read in interim typed data file
typed_hmda_data = fu.load_parquet("hmda_2024_typed")
print(typed_hmda_data.shape)

Loading dataset from /Users/c1burns/Documents/UTD/BUAN 6341/project_repo/data/interim/hmda_2024_typed.parquet
(8841112, 88)


In [10]:
# Generate multi-hot features for race/ethnicity features
feh.generate_multi_hot_features(typed_hmda_data, cfg_feature_engineering, "applicant_ethnicity_")
feh.generate_multi_hot_features(typed_hmda_data, cfg_feature_engineering, "co_applicant_ethnicity_")
feh.generate_multi_hot_features(typed_hmda_data, cfg_feature_engineering, "applicant_race_")
feh.generate_multi_hot_features(typed_hmda_data, cfg_feature_engineering, "co_applicant_race_")
print("Converted multi-hot columns for race/ethnicity")

Converted multi-hot columns for race/ethnicity
